In [ ]:
import sys, os
from sagemaker import image_uris

# Get the absolute path of the root directory
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.insert(0, root_dir)

## Local Mode ModelTrainer

In local mode training, user will train their model in a container that runs in their local machine.
You don't need to access any AWS resources unless you want to use data from S3 as input, or pull images from ECR.

## Simple Case Minimally Setup Local ModelTrainer and Execute Commands
When running model trainer in local mode, you need to have docker engine running in your environment.
When you run the following cell for the first time, a SageMaker session will be initiated to pull the image from ECR.
Once the image has been pulled, there won't be any AWS API call.

In [ ]:
from sagemaker.modules.train.model_trainer import ModelTrainer, Mode
from sagemaker.modules.configs import SourceCode
from sagemaker.modules.constants import DEFAULT_INSTANCE_TYPE

hugging_face_image = "763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04"

source_code = SourceCode(
    command="echo 'Hello World' && env",
)
model_trainer = ModelTrainer(
    training_image=hugging_face_image,
    source_code=source_code,
    training_mode=Mode.LOCAL_CONTAINER,
)

In [ ]:
model_trainer.train()

## Simple Script Mode Case - 1: Training with Local Data
In this example, everything (input, output, training resource) will be in your local environment. You don't need to use your AWS account at all for this.

In [ ]:
from sagemaker.modules.configs import Compute, InputData, SourceCode

source_code = SourceCode(
    source_dir="basic-script-mode",
    entry_script="local_training_script.py",
)

compute = Compute(
    instance_type="local_cpu",
    instance_count=1,
)

train_data = InputData(
    channel_name="train",
    data_source="basic-script-mode/data/train/",
)

test_data = InputData(
    channel_name="test",
    data_source="basic-script-mode/data/test/",
)

model_trainer = ModelTrainer(
    training_image=hugging_face_image,
    source_code=source_code,
    compute=compute,
    input_data_config=[train_data, test_data],
    base_job_name="local_mode_single_container_case_1",
    training_mode=Mode.LOCAL_CONTAINER,
)

In [ ]:
model_trainer.train()

## Simple Script Mode Case - 2: Training with Input Data from S3
In this example, the input data is read from S3. You will have to configure your AWS credentials before running this.

In [ ]:
train_data = InputData(
    channel_name="train", data_source="s3://morpheus-bugbash/basic-script-mode/data/train/"
)

test_data = InputData(
    channel_name="test", data_source="s3://morpheus-bugbash/basic-script-mode/data/test/"
)

model_trainer = ModelTrainer(
    training_image=hugging_face_image,
    source_code=source_code,
    compute=compute,
    input_data_config=[train_data, test_data],
    base_job_name="local_mode_single_container_case_2",
    training_mode=Mode.LOCAL_CONTAINER,
)

In [ ]:
model_trainer.train()